<a href="https://colab.research.google.com/github/dwikagt/P2P_byProfit/blob/main/Copy_of_Minggu12_lgbm_profitscore_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install -q hvplot
#!pip install tensorflow==2.7.0
# install the Kaggle API client
! pip install -q kaggle

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from scipy import stats 
#import hvplot.pandas
#Kaggle
from google.colab import files
import zipfile,os
#processing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
#metric performa
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import cross_val_score, cross_val_predict
from scipy import stats
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
pd.set_option('display.float', '{:.2f}'.format)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

In [3]:
# install the Kaggle API client
! pip install -q kaggle

In [4]:
# Unggah file kaggle.json
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"tiaralailatulnikmah","key":"1a294e2d607cef4fb50b5cd7366998e0"}'}

In [5]:
#membuat directori
!mkdir -p ~/.kaggle 
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d muchazizmuslim/p2p-lending-profit-score-dataset

100% 478M/480M [00:03<00:00, 200MB/s]
100% 480M/480M [00:03<00:00, 160MB/s]


In [7]:
# melakukan unzip pada dataset yang telah di download
import zipfile,os
local_zip = '/content/p2p-lending-profit-score-dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [8]:
#untuk membaca file
import pandas as pd
df = pd.read_csv ("/tmp/P2P-Lending-Profit-Score-Dataset.csv")

In [9]:
# The length of the data
print(f"The Length of the data: {df.shape}")

The Length of the data: (4303450, 80)


In [10]:
df['Profit_score'].value_counts()

1    2151725
0    2151725
Name: Profit_score, dtype: int64

In [11]:
n_sampling = 10000

profit_status_0 = df[df['Profit_score']==0]
profit_status_1 = df[df['Profit_score']==1]

subset_profit_status_0 = profit_status_0.sample(n=n_sampling, random_state=42)
subset_profit_status_1 = profit_status_1.sample(n=n_sampling, random_state=42)

print('subset_profit_status_0: ', subset_profit_status_0.shape)
print('subset_profit_status_1: ', subset_profit_status_1.shape)

subset_profit_status_0:  (10000, 80)
subset_profit_status_1:  (10000, 80)


In [12]:
df_sampler = pd.concat([subset_profit_status_0, subset_profit_status_1])

df_sampler = df_sampler.sample(frac=1, random_state=42). reset_index(drop=True)

print('df_sampler shape:', df_sampler.shape)
df_sampler.head()

df_sampler shape: (20000, 80)


,Unnamed: 0,loan_amnt,funded_amnt,term,int_rate,installment,annual_inc,dti,fico_range_low,fico_range_high,open_acc,pub_rec,revol_bal,revol_util,total_acc,last_pymnt_amnt,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mort_acc,num_actv_rev_tl,pub_rec_bankruptcies,...,sub_grade_G2,sub_grade_G3,sub_grade_G4,sub_grade_G5,verification_status_Source Verified,verification_status_Verified,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,initial_list_status_w,application_type_Joint App,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,Profit_score
0,1680305,12000.00,12000.00,60,15.99,291.76,78000.00,14.09,685.00,689.00,8.00,0,29221.00,78.30,10.00,10388.14,4.00,21852.00,3687.00,78.70,61.00,5.00,1.00,5.00,0.00,...,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,3936342,20104.54,20104.54,60,25.84,600.06,65387.19,13.20,669.65,673.65,12.54,0,12120.53,72.72,24.46,743.73,8.07,13314.32,945.44,93.22,207.36,11.85,1.00,3.46,0.00,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,4264169,14524.65,14524.65,36,30.84,617.06,116000.00,16.01,675.00,679.00,12.31,0,21239.67,81.86,37.76,14991.93,7.00,27322.91,739.17,96.01,120.90,25.13,1.00,7.17,0.00,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4268687,35000.00,35000.00,60,25.43,1036.13,102634.37,19.37,660.00,664.00,11.86,0,17201.86,80.02,27.41,25136.04,10.45,20870.08,3486.59,82.83,167.46,6.63,1.00,5.23,0.00,...,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,339757,27450.00,27450.00,36,16.99,978.54,61092.00,35.24,680.00,684.00,16.00,0,67383.00,67.00,42.00,2945.76,9.00,9180.00,8883.00,67.00,237.00,1.00,1.00,14.00,0.00,...,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [13]:
df_sampler['Profit_score'].value_counts()

1    10000
0    10000
Name: Profit_score, dtype: int64

In [14]:
!pip install sklearn-genetic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 15.9 MB/s eta 0:00:00


In [15]:
from genetic_selection import GeneticSelectionCV

In [16]:
X = df_sampler.drop(['Profit_score'], axis=1)
y = df_sampler['Profit_score']

In [17]:
columns_list = list(X.columns)

estimator = DecisionTreeClassifier(random_state=42)
selector = GeneticSelectionCV(estimator,cv=10,
                                  verbose=1,
                                  scoring="accuracy", 
                                  max_features=77,
                                  n_population=25,
                                  crossover_proba=1,
                                  mutation_proba=0.1,
                                  n_generations=50,
                                  crossover_independent_proba=0.1,
                                  mutation_independent_proba=0.01,
                                  n_gen_no_change=100,
                                  caching=True,
                                  n_jobs=-1)
model = selector.fit(X, y)
print('Features:', X.columns[model.support_])

Selecting features with genetic algorithm.
gen	nevals	avg                            	std                               	min                      	max                               
0  	25    	[  0.96338  42.8       0.00217]	[  0.09539   23.532106   0.00395 ]	[ 0.5927  6.      0.    ]	[  1.        77.         0.018762]
1  	24    	[-399.040866   61.4       400.000322]	[ 1959.787577    17.165081  1959.591728]	[-10000.     21.      0.]	[     1.     78.  10000.]         
2  	24    	[  0.99998   71.36       0.000023]   	[ 0.000069  4.107359  0.000079]         	[  0.9997  63.       0.    ]	[  1.        76.         0.000332]
3  	24    	[  1.    67.28   0.  ]               	[ 0.        3.013569  0.      ]         	[  1.  62.   0.]            	[  1.  72.   0.]                  
4  	24    	[  1.    64.92   0.  ]               	[ 0.        2.115089  0.      ]         	[  1.  62.   0.]            	[  1.  70.   0.]                  
5  	24    	[  1.    63.32   0.  ]               	[ 0.        1.256